# Profitable App Profiles for iOS/Android

Our aim in this project is to find mobile app profiles that are profitable for iOS and Android. As data analysts, our job is to enable a team of developers to make data-driven decisions with respect to the kind of apps they build next.

If the company only build apps that are free to download and install, their main source of revenue consists of in-app ads. This means that revenue for any given app is mostly influenced by the number of users that use the app. Our goal for this project is to analyze data to help developers understand what kinds of apps are likely to attract more users.

## Opening and Exploring the Data
Collecting data for over four million apps requires a significant amount of time and money, so we'll try to analyze a sample of data instead. To avoid spending resources with collecting new data ourselves, we should first try to see whether we can find any relevant existing data at no cost. Luckily, these are two data sets that seem suitable for our purpose:
- A data set containing data about approximately ten thousand Android apps from Google Play.
- A data set containing data about approximately seven thousand iOS apps from the App Store.

In [2]:
import pandas as pd

def open_data(url):
    df = pd.read_csv(url)
    return df

def explore_data(dataset, start, end, rows_and_columns=False):
    dataset_slice = dataset[start:end]    
    for row in dataset_slice:
        print(row)
        print('\n')

    if rows_and_columns:
        print('Number of rows:', len(dataset))
        print('Number of columns:', len(dataset[0]))
        
apple_url = 'https://dq-content.s3.amazonaws.com/350/AppleStore.csv'
google_url = 'https://dq-content.s3.amazonaws.com/350/googleplaystore.csv'

apple_df = open_data(apple_url)
google_df = open_data(google_url)

In [3]:
apple_df[:2]

,id,track_name,size_bytes,currency,price,rating_count_tot,rating_count_ver,user_rating,user_rating_ver,ver,cont_rating,prime_genre,sup_devices.num,ipadSc_urls.num,lang.num,vpp_lic
0,284882215,Facebook,389879808,USD,0.0,2974676,212,3.5,3.5,95.0,4+,Social Networking,37,1,29,1
1,389801252,Instagram,113954816,USD,0.0,2161558,1289,4.5,4.0,10.23,12+,Photo & Video,37,0,29,1


In [4]:
apple_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7197 entries, 0 to 7196
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id                7197 non-null   int64  
 1   track_name        7197 non-null   object 
 2   size_bytes        7197 non-null   int64  
 3   currency          7197 non-null   object 
 4   price             7197 non-null   float64
 5   rating_count_tot  7197 non-null   int64  
 6   rating_count_ver  7197 non-null   int64  
 7   user_rating       7197 non-null   float64
 8   user_rating_ver   7197 non-null   float64
 9   ver               7197 non-null   object 
 10  cont_rating       7197 non-null   object 
 11  prime_genre       7197 non-null   object 
 12  sup_devices.num   7197 non-null   int64  
 13  ipadSc_urls.num   7197 non-null   int64  
 14  lang.num          7197 non-null   int64  
 15  vpp_lic           7197 non-null   int64  
dtypes: float64(3), int64(8), object(5)
memory 

We have 7197 iOS apps in this data set, and the columns that seem interesting are: 'track_name', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', and 'prime_genre'.

In [5]:
google_df[:2]

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up


In [6]:
google_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10841 entries, 0 to 10840
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   App             10841 non-null  object 
 1   Category        10841 non-null  object 
 2   Rating          9367 non-null   float64
 3   Reviews         10841 non-null  object 
 4   Size            10841 non-null  object 
 5   Installs        10841 non-null  object 
 6   Type            10840 non-null  object 
 7   Price           10841 non-null  object 
 8   Content Rating  10840 non-null  object 
 9   Genres          10841 non-null  object 
 10  Last Updated    10841 non-null  object 
 11  Current Ver     10833 non-null  object 
 12  Android Ver     10838 non-null  object 
dtypes: float64(1), object(12)
memory usage: 1.1+ MB


We see that the Google Play data set has 10841 apps and 13 columns. At a quick glance, the columns that might be useful for the purpose of our analysis are 'App', 'Category', 'Reviews', 'Installs', 'Type', 'Price', and 'Genres'.

## Deleting Wrong Data

The Google Play data set has a dedicated discussion section, and we can see that [one of the discussions] https://www.kaggle.com/lava18/google-play-store-apps/discussion/66015 outlines an error for row 10472. Let's print this row and compare it against the header and another row that is correct.

In [7]:
google_df.iloc[10472]

App               Life Made WI-Fi Touchscreen Photo Frame
Category                                              1.9
Rating                                               19.0
Reviews                                              3.0M
Size                                               1,000+
Installs                                             Free
Type                                                    0
Price                                            Everyone
Content Rating                                        NaN
Genres                                  February 11, 2018
Last Updated                                       1.0.19
Current Ver                                    4.0 and up
Android Ver                                           NaN
Name: 10472, dtype: object

The row 10472 corresponds to the app Life Made WI-Fi Touchscreen Photo Frame, and we can see that the rating is 19. This is clearly off because the maximum rating for a Google Play app is 5 (as mentioned in the discussions section, this problem is caused by a missing value in the 'Category' column). As a consequence, we'll delete this row.

In [8]:
gdf_clean = google_df.drop(10472)
print(len(gdf_clean))

10840



## Removing Duplicate Entries
### Part One

If we explore the Google Play data set long enough, we'll find that some apps have more than one entry. For instance, the application Instagram has four entries:


In [24]:
for app in gdf_clean:
    print(app)
    # if name == 'Instagram':

App
Category
Rating
Reviews
Size
Installs
Type
Price
Content Rating
Genres
Last Updated
Current Ver
Android Ver


In total, there are 1,181 cases where an app occurs more than once:

In [10]:
duplicate_apps = []
unique_apps = []

for app in gdf_clean.iterrows():
    name = app[1].iloc[0]
    if name in unique_apps:
        duplicate_apps.append(name)
    else:
        unique_apps.append(name)

print('Number of duplicate apps:', len(duplicate_apps))
print('\n')
print('Examples of duplicate apps:', duplicate_apps[:15])

Number of duplicate apps: 1181


Examples of duplicate apps: ['Quick PDF Scanner + OCR FREE', 'Box', 'Google My Business', 'ZOOM Cloud Meetings', 'join.me - Simple Meetings', 'Box', 'Zenefits', 'Google Ads', 'Google My Business', 'Slack', 'FreshBooks Classic', 'Insightly CRM', 'QuickBooks Accounting: Invoicing & Expenses', 'HipChat - Chat Built for Teams', 'Xero Accounting Software']


We don't want to count certain apps more than once when we analyze data, so we need to remove the duplicate entries and keep only one entry per app. One thing we could do is remove the duplicate rows randomly, but we could probably find a better way.

If you examine the rows we printed two cells above for the Instagram app, the main difference happens on the fourth position of each row, which corresponds to the number of reviews. The different numbers show that the data was collected at different times. We can use this to build a criterion for keeping rows. We won't remove rows randomly, but rather we'll keep the rows that have the highest number of reviews because the higher the number of reviews, the more reliable the ratings.

To do that, we will:

- Create a dictionary where each key is a unique app name, and the value is the highest number of reviews of that app
- Use the dictionary to create a new data set, which will have only one entry per app (and we only select the apps with the highest number of reviews)

## Part Two
Let's start by building the dictionary.

In [11]:
reviews_max = {}

for app in gdf_clean.iterrows():
    name = app[1].iloc[0]
    n_reviews = float(app[1].iloc[3])
    
    if name in reviews_max and reviews_max[name] < n_reviews:
        reviews_max[name] = n_reviews
        
    elif name not in reviews_max:
        reviews_max[name] = n_reviews

In a previous code cell, we found that there are 1,181 cases where an app occurs more than once, so the length of our dictionary (of unique apps) should be equal to the difference between the length of our data set and 1,181.

In [12]:
print('Expected length:', len(gdf_clean) - 1181)
print('Actual length:', len(reviews_max))

Expected length: 9659
Actual length: 9659




Now, let's use the reviews_max dictionary to remove the duplicates. For the duplicate cases, we'll only keep the entries with the highest number of reviews. In the code cell below:

- We start by initializing two empty lists, android_clean and already_added.
- We loop through the android data set, and for every iteration:
    - We isolate the name of the app and the number of reviews.
    - We add the current row (app) to the android_clean list, and the app name (name) to the already_added list if:
        - The number of reviews of the current app matches the number of reviews of that app as described in the reviews_max dictionary; and
        - The name of the app is not already in the already_added list. We need to add this supplementary condition to account for those cases where the highest number of reviews of a duplicate app is the same for more than one entry (for example, the Box app has three entries, and the number of reviews is the same). If we just check for reviews_max[name] == n_reviews, we'll still end up with duplicate entries for some apps.

In [13]:
android_clean = []
already_added = []

for app in gdf_clean.iterrows():
    name = app[1].iloc[0]
    n_reviews = float(app[1].iloc[3])
    
    if (reviews_max[name] == n_reviews) and (name not in already_added):
        android_clean.append(app[1])
        already_added.append(name) # make sure this is inside the if block

In [14]:
explore_data(android_clean, 0, 3, True)

App               Photo Editor & Candy Camera & Grid & ScrapBook
Category                                          ART_AND_DESIGN
Rating                                                       4.1
Reviews                                                      159
Size                                                         19M
Installs                                                 10,000+
Type                                                        Free
Price                                                          0
Content Rating                                          Everyone
Genres                                              Art & Design
Last Updated                                     January 7, 2018
Current Ver                                                1.0.0
Android Ver                                         4.0.3 and up
Name: 0, dtype: object


App               U Launcher Lite – FREE Live Cool Themes, Hide ...
Category                                             ART_AND_D

## Removing Non-English Apps
### Part One

If you explore the data sets enough, you'll notice the names of some of the apps suggest they are not directed toward an English-speaking audience. Below, we see a couple of examples from both data sets:

In [15]:
print(apple_df.iloc[813])
print(apple_df.iloc[6731])

id                            445375097
track_name          爱奇艺PPS -《欢乐颂2》电视剧热播
size_bytes                    224617472
currency                            USD
price                               0.0
rating_count_tot                  14844
rating_count_ver                      0
user_rating                         4.0
user_rating_ver                     0.0
ver                               6.3.3
cont_rating                         17+
prime_genre               Entertainment
sup_devices.num                      38
ipadSc_urls.num                       5
lang.num                              3
vpp_lic                               1
Name: 813, dtype: object
id                                           1120021683
track_name          【脱出ゲーム】絶対に最後までプレイしないで 〜謎解き＆ブロックパズル〜
size_bytes                                     77551616
currency                                            USD
price                                               0.0
rating_count_tot                                      0

In [16]:
print(android_clean[4412])
print(android_clean[7940])

App                 中国語 AQリスニング
Category                 FAMILY
Rating                      NaN
Reviews                      21
Size                        17M
Installs                 5,000+
Type                       Free
Price                         0
Content Rating         Everyone
Genres                Education
Last Updated      June 22, 2016
Current Ver               2.4.0
Android Ver          4.0 and up
Name: 5513, dtype: object
App               لعبة تقدر تربح DZ
Category                     FAMILY
Rating                          4.2
Reviews                         238
Size                           6.8M
Installs                    10,000+
Type                           Free
Price                             0
Content Rating             Everyone
Genres                    Education
Last Updated      November 18, 2016
Current Ver                 6.0.0.0
Android Ver              4.1 and up
Name: 9117, dtype: object


We're not interested in keeping these kind of apps, so we'll remove them. One way to go about this is to remove each app whose name contains a symbol that is not commonly used in English text — English text usually includes letters from the English alphabet, numbers composed of digits from 0 to 9, punctuation marks (., !, ?, ;, etc.), and other symbols (+, *, /, etc.).

All these characters that are specific to English texts are encoded using the ASCII standard. Each ASCII character has a corresponding number between 0 and 127 associated with it, and we can take advantage of that to build a function that checks an app name and tells us whether it contains non-ASCII characters.

We built this function below, and we use the built-in ord() function to find out the corresponding encoding number of each character.

In [17]:
def is_english(string):
    
    for character in string:
        if ord(character) > 127:
            return False
    
    return True

print(is_english('Instagram'))
print(is_english('爱奇艺PPS -《欢乐颂2》电视剧热播'))

True
False


The function seems to work fine, but some English app names use emojis or other symbols (™, — (em dash), – (en dash), etc.) that fall outside of the ASCII range. Because of this, we'll remove useful apps if we use the function in its current form.

In [18]:
print(is_english('Docs To Go™ Free Office Suite'))
print(is_english('Instachat 😜'))

print(ord('™'))
print(ord('😜'))

False
False
8482
128540


### Part Two

To minimize the impact of data loss, we'll only remove an app if its name has more than three non-ASCII characters:

In [19]:
def is_english(string):
    non_ascii = 0
    
    for character in string:
        if ord(character) > 127:
            non_ascii += 1
    
    if non_ascii > 3:
        return False
    else:
        return True

print(is_english('Docs To Go™ Free Office Suite'))
print(is_english('Instachat 😜'))

True
True


The function is still not perfect, and very few non-English apps might get past our filter, but this seems good enough at this point in our analysis — we shouldn't spend too much time on optimization at this point.

Below, we use the is_english() function to filter out the non-English apps for both data sets:

In [20]:
android_english = []
ios_english = []

for app in android_clean:
    name = app[0]
    if is_english(name):
        android_english.append(app[1])
        
for app in apple_df:
    name = app[1]
    if is_english(name):
        ios_english.append(app)
        
explore_data(android_english, 0, 3, True)
print('\n')
explore_data(ios_english, 0, 3, True)

ART_AND_DESIGN


ART_AND_DESIGN


ART_AND_DESIGN


Number of rows: 9614
Number of columns: 14


id


track_name


size_bytes


Number of rows: 16
Number of columns: 2
